<a href="https://colab.research.google.com/github/shrisha337-beep/Library-Slip-Tracker/blob/main/Library_slip_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Import Libraries

In [4]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



Step 2: Generate Synthetic Dataset

In [5]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta, datetime

#Setup
np.random.seed(42)
student_names = ['Raghav Singh','Rahul Sharma','Anjali Khanna','Khushi Mukhija','Shreeja Chaturvedi','Megha Jain','Kunal Mehta','Ananya Shroff','Karan Ahuja','Sneha Kapoor',
                 'Rohit Kashyap','Megha Jain','Vivek Agrawal','Isha Khanna','Aman Verma','Priya Desai','Shreya Sharma']
book_titles = ['Introduction to ML','Python Basics','Data Science with Python','Modern Web-Dev','Operating Systems','Linear Algebra','Digital Circutis',
                'Database Systems','Advanced Machine Learning','Artificial Intelligence','Electronic Circuits']
genres = ['CS','Math','Electronics','AI','Web Dev']
librarians = ['Mrs. Batra','Mr. Khurana','Ms. Sheikh','Mrs. Kapoor','Mr. Ahuja']

#Settings
n = 500
fine_per_day = 1
start_date = datetime(2025,1,1)

#17 offsets from -2 to 14
return_offsets = list(range(-2,15))
#17 probabilities matching above
probabilities = np.array([0.03, 0.03, 0.06, 0.06, 0.06, 0.06, 0.06,
                          0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05,
                          0.05, 0.05, 0.05])
#Normalize to sum to 1.0
probabilities = probabilities/probabilities.sum()

#Generate Data
data = []
for i in range(n):
    slip_id = f"SLIP{i+1:05d}"
    student = random.choice(student_names)
    book = random.choice(book_titles)
    genre = random.choice(genres)
    librarian = random.choice(librarians)

    issue_date = start_date + timedelta(days=random.randint(0,365))
    due_date = issue_date + timedelta(days=14) # Due date should be relative to issue date
    return_offset = random.choices(return_offsets, weights=probabilities, k=1)[0]
    return_date = issue_date + timedelta(days=int(return_offset))

    delay_days = max(0,(return_date - due_date).days)
    fine_amount = delay_days * fine_per_day

    data.append([slip_id, student, book, genre, librarian, issue_date, due_date, return_date, delay_days, fine_amount])

# Create and Save Dataframe
columns = ['Slip_Id','Student_Name','Book_Title','Genre','Librarian_Name','Issue_Date','Due_Date','Return_Date','Delay_Days','Fine']
df = pd.DataFrame(data,columns=columns)
df.to_csv('library_slip_tracker.csv',index=False)
print("Synthetic dataset generated and saved as 'library_slip_tracker.csv'")

#Display first 10 rows
print(df.head(499))
df = pd.read_csv('library_slip_tracker.csv')
df.to_excel('library_slip_tracker.xlsx',index=False)

Synthetic dataset generated and saved as 'library_slip_tracker.csv'
       Slip_Id        Student_Name                 Book_Title        Genre  \
0    SLIP00001       Shreya Sharma  Advanced Machine Learning           AI   
1    SLIP00002         Isha Khanna          Operating Systems      Web Dev   
2    SLIP00003       Rohit Kashyap    Artificial Intelligence         Math   
3    SLIP00004         Isha Khanna  Advanced Machine Learning           CS   
4    SLIP00005         Priya Desai          Operating Systems  Electronics   
..         ...                 ...                        ...          ...   
494  SLIP00495          Megha Jain             Modern Web-Dev           CS   
495  SLIP00496  Shreeja Chaturvedi              Python Basics           AI   
496  SLIP00497      Khushi Mukhija           Database Systems      Web Dev   
497  SLIP00498        Raghav Singh        Electronic Circuits         Math   
498  SLIP00499          Megha Jain    Artificial Intelligence           CS

Step 3: Regression Model Training

In [6]:
# Extract DayofWeek from 'Issue_Date'
df['Issue_Date'] = pd.to_datetime(df['Issue_Date'])
df['DayofWeek'] = df['Issue_Date'].dt.dayofweek

# Add a placeholder for 'TotalStudents' - you might want to replace this with a more meaningful feature
df['TotalStudents'] = 500 # Placeholder value

X = df[["DayofWeek","TotalStudents"]]
Y = df["Fine"]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

model=RandomForestRegressor(n_estimators=100,random_state=42)
model.fit(X_train,Y_train)
y_pred=model.predict(X_test)

future_data = pd.DataFrame({
    "DayofWeek": [0],  # Monday
    "TotalStudents": [1]
})
predicted_fine = model.predict(future_data)
print(f"Predicted fine for 1 students on Monday: ₹{predicted_fine[0]:.2f}")

Predicted fine for 1 students on Monday: ₹0.00


Step 4: Evaluation

In [7]:
mae = mean_absolute_error(Y_test,y_pred)
mse = mean_squared_error(Y_test,y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(Y_test,y_pred)

print(f"Mean Absolute Error: ₹{mae: .2f}")
print(f"Mean Squared Error: ₹{mse: .2f}")
print(f"Root Mean Squared Error: ₹{rmse: .2f}")
print(f"R-squared: {r2: .2f}")


Mean Absolute Error: ₹ 0.00
Mean Squared Error: ₹ 0.00
Root Mean Squared Error: ₹ 0.00
R-squared:  1.00


Step 5: Future Fine Details

In [8]:
import pandas as pd

# The prediction code has been moved to the previous cell (id: 67Dmpn5HaCb2) where the model is defined and trained.
# You can now run that cell to see the prediction for future_data.